In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from utils import make_regression_data, mse, log_epoch, RegressionDataset

class LinearRegression(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.linear = nn.Linear(in_features, 1)

    def forward(self, x):
        return self.linear(x)

if __name__ == '__main__':
    # Генерируем данные
    X, y = make_regression_data(n=200)
    
    # Создаём датасет и даталоадер
    dataset = RegressionDataset(X, y)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
    print(f'Размер датасета: {len(dataset)}')
    print(f'Количество батчей: {len(dataloader)}')
    
    # Создаём модель, функцию потерь и оптимизатор
    model = LinearRegression(in_features=1)
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.1)
    
    reg_type = 'l2'  # или 'l1', или None
    alpha = 0.0001    # Коэффициент регуляризации

    d = 0.001 #коэфицент остановки
    prev_weights = [w.detach().clone() for w in model.linear.weight] #сохраняем веса 

    # Обучаем модель
    epochs = 100
    for epoch in range(1, epochs + 1):
        total_loss = 0
        
        for i, (batch_X, batch_y) in enumerate(dataloader):
            optimizer.zero_grad()
            y_pred = model(batch_X)
            reg = 0
            if reg_type == 'l1':
                reg = alpha * model.linear.weight.abs().sum() #l1
            elif reg_type == 'l2':
                reg = alpha * (model.linear.weight ** 2).sum() #l2
            loss = criterion(y_pred, batch_y) + reg #штраф
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()

        weight_change = ((model.linear.weight - prev_weights[0]) ** 2).sum().sqrt().item() #проверка разницы весов с прошлой итерацией
        if weight_change < d:
            print(f"Остановка перед переобучением")
            break
        prev_weights = [w.detach().clone() for w in model.linear.weight] #сохраняем веса итерации

        avg_loss = total_loss / (i + 1)
        if epoch % 10 == 0:
            log_epoch(epoch, avg_loss)

    # Сохраняем модель
    torch.save(model.state_dict(), 'linreg_torch.pth')
    
    # Загружаем модель
    new_model = LinearRegression(in_features=1)
    new_model.load_state_dict(torch.load('linreg_torch.pth'))
    new_model.eval() 